In [75]:
import re
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime, timedelta
import pandas_datareader as wp
import yfinance as yf

pd.options.display.max_columns = None

In [76]:
# https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br
dataColumns =  pd.read_csv('statusinvest-busca-avancada-l.csv', sep=';',encoding='latin-1')
columns = dataColumns.columns
columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [84]:
cod = dataColumns['TICKER'].to_numpy()
len(cod)

293

In [85]:
cod

array(['AALR3', 'ABCB4', 'ABEV3', 'AERI3', 'AESB3', 'AGRO3', 'AGXY3',
       'ALLD3', 'ALPA4', 'ALPK3', 'ALSO3', 'ALUP11', 'AMAR3', 'AMBP3',
       'AMER3', 'ANIM3', 'APER3', 'ARML3', 'ARZZ3', 'ASAI3', 'AURA33',
       'AURE3', 'AZEV3', 'AZEV4', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3',
       'BBDC4', 'BBSE3', 'BEEF3', 'BIDI11', 'BIDI3', 'BIDI4', 'BKBR3',
       'BLAU3', 'BMGB4', 'BMOB3', 'BOAS3', 'BPAC11', 'BPAN4', 'BRAP3',
       'BRAP4', 'BRBI11', 'BRFS3', 'BRIT3', 'BRKM3', 'BRKM5', 'BRML3',
       'BRPR3', 'BRSR6', 'CAML3', 'CARD3', 'CASH3', 'CBAV3', 'CCRO3',
       'CEAB3', 'CESP6', 'CGRA4', 'CIEL3', 'CLSA3', 'CLSC4', 'CMIG3',
       'CMIG4', 'CMIN3', 'COCE5', 'COGN3', 'CPFE3', 'CPLE11', 'CPLE3',
       'CPLE6', 'CRFB3', 'CSAN3', 'CSED3', 'CSMG3', 'CSNA3', 'CURY3',
       'CVCB3', 'CXSE3', 'CYRE3', 'DASA3', 'DESK3', 'DEXP3', 'DIRR3',
       'DMMO3', 'DMVF3', 'DXCO3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6',
       'ELMD3', 'EMBR3', 'ENAT3', 'ENBR3', 'ENEV3', 'ENGI11', 'ENJU3',
       'EQTL3

In [86]:
#If error
cod = cod[131:]
count = 131
cod

array(['IGTI3', 'INEP3', 'INEP4', 'INTB3', 'IRBR3', 'ITSA3', 'ITSA4',
       'ITUB3', 'ITUB4', 'JALL3', 'JBSS3', 'JHSF3', 'JSLG3', 'KEPL3',
       'KLBN11', 'KLBN3', 'KLBN4', 'KRSA3', 'LAME3', 'LAME4', 'LAND3',
       'LAVV3', 'LCAM3', 'LEVE3', 'LIGT3', 'LJQQ3', 'LLIS3', 'LOGG3',
       'LOGN3', 'LPSB3', 'LREN3', 'LUPA3', 'LVTC3', 'LWSA3', 'MATD3',
       'MBLY3', 'MDIA3', 'MDNE3', 'MEAL3', 'MEGA3', 'MELK3', 'MGLU3',
       'MILS3', 'MLAS3', 'MOAR3', 'MODL11', 'MOSI3', 'MOVI3', 'MRFG3',
       'MRVE3', 'MTRE3', 'MULT3', 'MYPK3', 'NEOE3', 'NGRD3', 'NINJ3',
       'NTCO3', 'ODPV3', 'OFSA3', 'OIBR3', 'OIBR4', 'OMGE3', 'ONCO3',
       'OPCT3', 'ORVR3', 'PARD3', 'PCAR3', 'PDGR3', 'PETR3', 'PETR4',
       'PETZ3', 'PFRM3', 'PGMN3', 'PLPL3', 'PMAM3', 'PNVL3', 'POMO4',
       'PORT3', 'POSI3', 'POWE3', 'PRIO3', 'PRNR3', 'PSSA3', 'PTBL3',
       'QUAL3', 'RADL3', 'RAIL3', 'RAIZ4', 'RANI3', 'RAPT4', 'RCSL3',
       'RCSL4', 'RDOR3', 'RECV3', 'RENT3', 'RNEW3', 'RNEW4', 'ROMI3',
       'RRRP3', 'R

In [80]:
urlFundamental = 'https://finance.yahoo.com/quote/{0}.SA/financials?p={1}.SA'
count = 43

In [87]:
for stock in cod:
    # 2 Return HTML
    url = urlFundamental.format(stock,stock)

    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}
    resp = requests.get(url,headers= headers)
    print(url, resp, count)
    count = count + 1
    
    # 3 Filter page
    brute = BeautifulSoup(resp.text, "html.parser")
    pattern = re.compile(r'\s--\sData\s--\s')
        
    try:
        brute_datas = brute.find('script', text = pattern).contents[0]
        init = brute_datas.find("context")-2
        dados_json =json.loads(brute_datas[init:-12])
    except TypeError:
        continue
    except KeyError:
        continue
    except NoneType:
        continue

    # 4 Return Income Statement
    try:
        income_statement = dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistoryQuarterly']['incomeStatementHistory']
    except TypeError:
        continue
    except KeyError:
        continue
        
    financial_quarterly = []
    for s in income_statement:
        balance = {}
        for key, val in s.items():
            try:
                balance[key] = val["raw"]
            except TypeError:
                continue
            except KeyError:
                continue
        financial_quarterly.append(balance)


    annual_index = []
    try:
        annual_0 = pd.DataFrame.from_dict(financial_quarterly[0], orient="index").transpose()
        annual_0 = annual_0.rename(index={0: pd.to_datetime(annual_0["endDate"], unit='s')[0]}) 
        annual_index.append(pd.to_datetime(annual_0["endDate"], unit='s')[0])
        annual_1 = pd.DataFrame.from_dict(financial_quarterly[1], orient="index").transpose()
        annual_1 = annual_1.rename(index={0:  pd.to_datetime(annual_1["endDate"], unit='s')[0]}) 
        annual_index.append(pd.to_datetime(annual_1["endDate"], unit='s')[0])
        annual_2 = pd.DataFrame.from_dict(financial_quarterly[2], orient="index").transpose()
        annual_2 = annual_2.rename(index={0: pd.to_datetime(annual_2["endDate"], unit='s')[0]}) 
        annual_index.append(pd.to_datetime(annual_2["endDate"], unit='s')[0])
        annual_3 = pd.DataFrame.from_dict(financial_quarterly[3], orient="index").transpose()
        annual_3 = annual_3.rename(index={0:  pd.to_datetime(annual_3["endDate"], unit='s')[0]}) 
        annual_index.append(pd.to_datetime(annual_3["endDate"], unit='s')[0])
    except TypeError:
        continue
    except KeyError:
        continue
    except IndexError:
        continue

    income_statement = pd.concat([annual_0, annual_1, annual_2, annual_3])

    # 5 Balance sheet
    balance_sheet = dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistoryQuarterly']['balanceSheetStatements']

    balace_sheet_quarterly = []
    for s in balance_sheet:
        balance = {}
        for key, val in s.items():
            try:
                balance[key] = val["raw"]
            except TypeError:
                continue
            except KeyError:
                continue
        balace_sheet_quarterly.append(balance)

    try:
        bs_annual_0 = pd.DataFrame.from_dict(balace_sheet_quarterly[0], orient="index").transpose()
        bs_annual_0 = bs_annual_0.rename(index={0: pd.to_datetime(bs_annual_0["endDate"], unit='s')[0]}) 
        bs_annual_1 = pd.DataFrame.from_dict(balace_sheet_quarterly[1], orient="index").transpose()
        bs_annual_1 = bs_annual_1.rename(index={0: pd.to_datetime(bs_annual_1["endDate"], unit='s')[0]}) 
        bs_annual_2 = pd.DataFrame.from_dict(balace_sheet_quarterly[2], orient="index").transpose()
        bs_annual_2 = bs_annual_2.rename(index={0: pd.to_datetime(bs_annual_2["endDate"], unit='s')[0]}) 
        bs_annual_3 = pd.DataFrame.from_dict(balace_sheet_quarterly[3], orient="index").transpose()
        bs_annual_3 = bs_annual_3.rename(index={0: pd.to_datetime(bs_annual_3["endDate"], unit='s')[0]}) 
    except TypeError:
        continue
    except KeyError:
        continue
    except IndexError:
        continue

    bs_annual = pd.concat([bs_annual_0, bs_annual_1, bs_annual_2, bs_annual_3])

    ic_transpose = income_statement.transpose()
    bs_transpose = bs_annual.transpose()
    final = pd.concat([ic_transpose, bs_transpose])
    final = final.transpose()
        
    fileName = "data/{}.csv"
    fileName = fileName.format(stock)
    print(fileName)
        
    try:
        stockName = "{}.SA"
        stockName = stockName.format(stock)
        stock_0 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[0] - timedelta(days=15),end=annual_index[0] - timedelta(days=10))
        stock_0 = stock_0.drop(stock_0.columns[[0,1,2,3,4]], axis = 1)
        stock_1 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[1] - timedelta(days=15),end=annual_index[1] - timedelta(days=10))
        stock_1 = stock_1.drop(stock_1.columns[[0,1,2,3,4]], axis = 1)
        stock_2 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[2] - timedelta(days=15),end=annual_index[2] - timedelta(days=10))
        stock_2 = stock_2.drop(stock_2.columns[[0,1,2,3,4]], axis = 1)
        stock_3 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[3] - timedelta(days=15),end=annual_index[3] - timedelta(days=10))
        stock_3 = stock_3.drop(stock_3.columns[[0,1,2,3,4]], axis = 1)
    except TypeError:
        continue
    except KeyError:
        continue
    except RemoteDataError:
        continue
        
    try:
        amount_shares_0 = dados_json['context']['dispatcher']['stores']['QuoteTimeSeriesStore']["timeSeries"]['annualBasicAverageShares'][0]["reportedValue"]["raw"]
        final["amount_shares"] = [amount_shares_0, amount_shares_0, amount_shares_0, amount_shares_0]
    except TypeError:
        continue
    except KeyError:
        continue
    except IndexError:
        continue
        
    price_stock_0 = stock_0.iloc[0,0]
    price_stock_1 = stock_1.iloc[0,0]
    price_stock_2 = stock_2.iloc[0,0]
    price_stock_3 = stock_3.iloc[0,0]
        
    final["price_stock"] = [price_stock_0,price_stock_1,price_stock_2,price_stock_3]
    final["LPA"] = final["netIncomeApplicableToCommonShares"]/final["amount_shares"]
    final["preco_lucro"] = final["price_stock"]/final["LPA"]
    final["P/VP"] = final["price_stock"]*final["amount_shares"]
    final["PL"] = final["totalCurrentLiabilities"]*final["amount_shares"]
    final["P/ATIVO"] = final["price_stock"]/(final["totalAssets"]/final["amount_shares"])
    final["VPA"] = final["totalStockholderEquity"]/final["amount_shares"]
    final["VM"] = final["amount_shares"]*final["price_stock"]
    try:
        final["EV"] = final["VM"]+final["shortLongTermDebt"]-final["cash"]
        final["EV"] = final["VM"]+final["shortLongTermDebt"]-final["cash"]
        final["EV/EBIT"] = final["EV"]/final["ebit"]
        final["NOPAT"] = final["ebit"]-final["incomeTaxExpense"]
        final["ROIC-I"] = final["NOPAT"]/final["longTermInvestments"]
    except TypeError:
        final.to_csv(fileName)
    except KeyError:
        final.to_csv(fileName)
    final["formula-gharant"] = (final["VPA"]*final["LPA"]*22.5)**0.5
    final["margem-seguranca"] = (final["formula-gharant"]/final["price_stock"]-1)*100

        
    final.to_csv(fileName)
            
        
    

https://finance.yahoo.com/quote/IGTI3.SA/financials?p=IGTI3.SA <Response [200]> 131
data/IGTI3.csv
https://finance.yahoo.com/quote/INEP3.SA/financials?p=INEP3.SA <Response [200]> 132
data/INEP3.csv
https://finance.yahoo.com/quote/INEP4.SA/financials?p=INEP4.SA <Response [200]> 133
data/INEP4.csv
https://finance.yahoo.com/quote/INTB3.SA/financials?p=INTB3.SA <Response [200]> 134
data/INTB3.csv
https://finance.yahoo.com/quote/IRBR3.SA/financials?p=IRBR3.SA <Response [200]> 135
data/IRBR3.csv
https://finance.yahoo.com/quote/ITSA3.SA/financials?p=ITSA3.SA <Response [200]> 136
data/ITSA3.csv
https://finance.yahoo.com/quote/ITSA4.SA/financials?p=ITSA4.SA <Response [200]> 137
data/ITSA4.csv
https://finance.yahoo.com/quote/ITUB3.SA/financials?p=ITUB3.SA <Response [200]> 138
data/ITUB3.csv
https://finance.yahoo.com/quote/ITUB4.SA/financials?p=ITUB4.SA <Response [200]> 139
data/ITUB4.csv
https://finance.yahoo.com/quote/JALL3.SA/financials?p=JALL3.SA <Response [200]> 140
data/JALL3.csv
https://fi

https://finance.yahoo.com/quote/QUAL3.SA/financials?p=QUAL3.SA <Response [200]> 215
data/QUAL3.csv
https://finance.yahoo.com/quote/RADL3.SA/financials?p=RADL3.SA <Response [200]> 216
data/RADL3.csv
https://finance.yahoo.com/quote/RAIL3.SA/financials?p=RAIL3.SA <Response [200]> 217
data/RAIL3.csv
https://finance.yahoo.com/quote/RAIZ4.SA/financials?p=RAIZ4.SA <Response [200]> 218
data/RAIZ4.csv
https://finance.yahoo.com/quote/RANI3.SA/financials?p=RANI3.SA <Response [200]> 219
data/RANI3.csv
https://finance.yahoo.com/quote/RAPT4.SA/financials?p=RAPT4.SA <Response [200]> 220
data/RAPT4.csv
https://finance.yahoo.com/quote/RCSL3.SA/financials?p=RCSL3.SA <Response [200]> 221
data/RCSL3.csv
https://finance.yahoo.com/quote/RCSL4.SA/financials?p=RCSL4.SA <Response [200]> 222
data/RCSL4.csv
https://finance.yahoo.com/quote/RDOR3.SA/financials?p=RDOR3.SA <Response [200]> 223
data/RDOR3.csv
https://finance.yahoo.com/quote/RECV3.SA/financials?p=RECV3.SA <Response [200]> 224
data/RECV3.csv
https://fi

In [32]:
for stock in cod:
    try:
         # 2 Return HTML
        url = urlFundamental.format(stock,stock)

        headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}
        resp = requests.get(url,headers= headers)
        print(url, resp)
        # 3 Filter page
        try:
            brute = BeautifulSoup(resp.text, "html.parser")
            pattern = re.compile(r'\s--\sData\s--\s')
            try:
                brute_datas = brute.find('script', text = pattern).contents[0]
            except TypeError:
                continue
            except KeyError:
                continue

            init = brute_datas.find("context")-2
            dados_json =json.loads(brute_datas[init:-12])

            # 4 Return Income Statement
            income_statement = dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
            financial_quarterly = []
            for s in income_statement:
                balance = {}
                for key, val in s.items():
                    try:
                        balance[key] = val["raw"]
                    except TypeError:
                        continue
                    except KeyError:
                        continue
                financial_quarterly.append(balance)


            annual_index = []
            annual_0 = pd.DataFrame.from_dict(financial_quarterly[0], orient="index").transpose()
            annual_0 = annual_0.rename(index={0: pd.to_datetime(annual_0["endDate"], unit='s')[0]}) 
            annual_index.append(pd.to_datetime(annual_0["endDate"], unit='s')[0])
            annual_1 = pd.DataFrame.from_dict(financial_quarterly[1], orient="index").transpose()
            annual_1 = annual_1.rename(index={0:  pd.to_datetime(annual_1["endDate"], unit='s')[0]}) 
            annual_index.append(pd.to_datetime(annual_1["endDate"], unit='s')[0])
            annual_2 = pd.DataFrame.from_dict(financial_quarterly[2], orient="index").transpose()
            annual_2 = annual_2.rename(index={0: pd.to_datetime(annual_2["endDate"], unit='s')[0]}) 
            annual_index.append(pd.to_datetime(annual_2["endDate"], unit='s')[0])
            annual_3 = pd.DataFrame.from_dict(financial_quarterly[3], orient="index").transpose()
            annual_3 = annual_3.rename(index={0:  pd.to_datetime(annual_3["endDate"], unit='s')[0]}) 
            annual_index.append(pd.to_datetime(annual_3["endDate"], unit='s')[0])

            income_statement = pd.concat([annual_0, annual_1, annual_2, annual_3])

            # 5 Balance sheet
            balance_sheet = dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
            balace_sheet_quarterly = []
            for s in balance_sheet:
                balance = {}
                for key, val in s.items():
                    try:
                        balance[key] = val["raw"]
                    except TypeError:
                        continue
                    except KeyError:
                        continue
                balace_sheet_quarterly.append(balance)

            bs_annual_0 = pd.DataFrame.from_dict(balace_sheet_quarterly[0], orient="index").transpose()
            bs_annual_0 = bs_annual_0.rename(index={0: pd.to_datetime(bs_annual_0["endDate"], unit='s')[0]}) 
            bs_annual_1 = pd.DataFrame.from_dict(balace_sheet_quarterly[1], orient="index").transpose()
            bs_annual_1 = bs_annual_1.rename(index={0: pd.to_datetime(bs_annual_1["endDate"], unit='s')[0]}) 
            bs_annual_2 = pd.DataFrame.from_dict(balace_sheet_quarterly[2], orient="index").transpose()
            bs_annual_2 = bs_annual_2.rename(index={0: pd.to_datetime(bs_annual_2["endDate"], unit='s')[0]}) 
            bs_annual_3 = pd.DataFrame.from_dict(balace_sheet_quarterly[3], orient="index").transpose()
            bs_annual_3 = bs_annual_3.rename(index={0: pd.to_datetime(bs_annual_3["endDate"], unit='s')[0]}) 

            bs_annual = pd.concat([bs_annual_0, bs_annual_1, bs_annual_2, bs_annual_3])

            ic_transpose = income_statement.transpose()
            bs_transpose = bs_annual.transpose()
            final = pd.concat([ic_transpose, bs_transpose])
            final = final.transpose()

            fileName = "data-annual/{}.csv"
            fileName = fileName.format(stock)
            print(fileName)
        except TypeError:
            continue
        except KeyError:
            continue
        except IndexError:
            continue
        except IndexError:
            continue
        
        try:
            stockName = "{}.SA"
            stockName = stockName.format(stock)
            stock_0 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[0] - timedelta(days=15),end=annual_index[0] - timedelta(days=10))
            stock_0 = stock_0.drop(stock_0.columns[[0,1,2,3,4]], axis = 1)
            stock_1 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[1] - timedelta(days=15),end=annual_index[1] - timedelta(days=10))
            stock_1 = stock_1.drop(stock_1.columns[[0,1,2,3,4]], axis = 1)
            stock_2 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[2] - timedelta(days=15),end=annual_index[2] - timedelta(days=10))
            stock_2 = stock_2.drop(stock_2.columns[[0,1,2,3,4]], axis = 1)
            stock_3 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[3] - timedelta(days=15),end=annual_index[3] - timedelta(days=10))
            stock_3 = stock_3.drop(stock_3.columns[[0,1,2,3,4]], axis = 1)
        except TypeError:
            continue
        except KeyError:
            continue
        except NoneType:
            continue
            
        
        try:
            amount_shares_0 = dados_json['context']['dispatcher']['stores']['QuoteTimeSeriesStore']["timeSeries"]['annualBasicAverageShares'][0]["reportedValue"]["raw"]
            final["amount_shares"] = [amount_shares_0, amount_shares_0, amount_shares_0, amount_shares_0]

            price_stock_0 = stock_0.iloc[0,0]
            price_stock_1 = stock_1.iloc[0,0]
            price_stock_2 = stock_2.iloc[0,0]
            price_stock_3 = stock_3.iloc[0,0]
            final["price_stock"] = [price_stock_0,price_stock_1,price_stock_2,price_stock_3]
        except TypeError:
            continue
        except KeyError:
            continue
        except IndexError:
            continue
         
        try:
            final["LPA"] = final["netIncomeApplicableToCommonShares"]/final["amount_shares"]
            final["preco_lucro"] = final["price_stock"]/final["LPA"]
            final["P/VP"] = final["price_stock"]*final["amount_shares"]
            final["PL"] = final["totalCurrentLiabilities"]*final["amount_shares"]
            final["P/ATIVO"] = final["price_stock"]/(final["totalAssets"]/final["amount_shares"])
            final["VPA"] = final["totalStockholderEquity"]/final["amount_shares"]
            final["VM"] = final["amount_shares"]*final["price_stock"]
            final["EV"] = final["VM"]+final["shortLongTermDebt"]-final["cash"]
            final["EV"] = final["VM"]+final["shortLongTermDebt"]-final["cash"]
            final["EV/EBIT"] = final["EV"]/final["ebit"]
            final["NOPAT"] = final["ebit"]-final["incomeTaxExpense"]
            final["ROIC"] = final["NOPAT"]/final["longTermInvestments"]
            final["formula-gharant"] = (final["VPA"]*final["LPA"]*22.5)**0.5
            final["margem-seguranca"] = (final["formula-gharant"]/final["price_stock"]-1)*100
        except TypeError:
            final.to_csv(fileName)
        except KeyError:
            final.to_csv(fileName)
        except IndexError:
            final.to_csv(fileName)
            
        
    
    except TypeError:
        continue
    except KeyError:
        continue
    except IndexError:
        continue

https://finance.yahoo.com/quote/AALR3.SA/financials?p=AALR3.SA <Response [200]>
data-annual/AALR3.csv
https://finance.yahoo.com/quote/ABCB4.SA/financials?p=ABCB4.SA <Response [200]>
data-annual/ABCB4.csv
https://finance.yahoo.com/quote/ABEV3.SA/financials?p=ABEV3.SA <Response [200]>
data-annual/ABEV3.csv
https://finance.yahoo.com/quote/AERI3.SA/financials?p=AERI3.SA <Response [200]>
data-annual/AERI3.csv
https://finance.yahoo.com/quote/AESB3.SA/financials?p=AESB3.SA <Response [200]>
https://finance.yahoo.com/quote/AGRO3.SA/financials?p=AGRO3.SA <Response [200]>
data-annual/AGRO3.csv
https://finance.yahoo.com/quote/AGXY3.SA/financials?p=AGXY3.SA <Response [200]>
data-annual/AGXY3.csv
https://finance.yahoo.com/quote/ALLD3.SA/financials?p=ALLD3.SA <Response [200]>
data-annual/ALLD3.csv
https://finance.yahoo.com/quote/ALPA4.SA/financials?p=ALPA4.SA <Response [200]>
data-annual/ALPA4.csv
https://finance.yahoo.com/quote/ALPK3.SA/financials?p=ALPK3.SA <Response [200]>
data-annual/ALPK3.csv
ht

https://finance.yahoo.com/quote/DEXP3.SA/financials?p=DEXP3.SA <Response [200]>
data-annual/DEXP3.csv
https://finance.yahoo.com/quote/DIRR3.SA/financials?p=DIRR3.SA <Response [200]>
data-annual/DIRR3.csv
https://finance.yahoo.com/quote/DMMO3.SA/financials?p=DMMO3.SA <Response [200]>
data-annual/DMMO3.csv
https://finance.yahoo.com/quote/DMVF3.SA/financials?p=DMVF3.SA <Response [200]>
data-annual/DMVF3.csv
https://finance.yahoo.com/quote/DXCO3.SA/financials?p=DXCO3.SA <Response [200]>
data-annual/DXCO3.csv
https://finance.yahoo.com/quote/ECOR3.SA/financials?p=ECOR3.SA <Response [200]>
data-annual/ECOR3.csv
https://finance.yahoo.com/quote/EGIE3.SA/financials?p=EGIE3.SA <Response [200]>
data-annual/EGIE3.csv
https://finance.yahoo.com/quote/ELET3.SA/financials?p=ELET3.SA <Response [200]>
data-annual/ELET3.csv
https://finance.yahoo.com/quote/ELET6.SA/financials?p=ELET6.SA <Response [200]>
data-annual/ELET6.csv
https://finance.yahoo.com/quote/ELMD3.SA/financials?p=ELMD3.SA <Response [200]>
da

https://finance.yahoo.com/quote/MATD3.SA/financials?p=MATD3.SA <Response [200]>
data-annual/MATD3.csv
https://finance.yahoo.com/quote/MBLY3.SA/financials?p=MBLY3.SA <Response [200]>
data-annual/MBLY3.csv
https://finance.yahoo.com/quote/MDIA3.SA/financials?p=MDIA3.SA <Response [200]>
data-annual/MDIA3.csv
https://finance.yahoo.com/quote/MDNE3.SA/financials?p=MDNE3.SA <Response [200]>
data-annual/MDNE3.csv
https://finance.yahoo.com/quote/MEAL3.SA/financials?p=MEAL3.SA <Response [200]>
data-annual/MEAL3.csv
https://finance.yahoo.com/quote/MEGA3.SA/financials?p=MEGA3.SA <Response [200]>
data-annual/MEGA3.csv
https://finance.yahoo.com/quote/MELK3.SA/financials?p=MELK3.SA <Response [200]>
data-annual/MELK3.csv
https://finance.yahoo.com/quote/MGLU3.SA/financials?p=MGLU3.SA <Response [200]>
data-annual/MGLU3.csv
https://finance.yahoo.com/quote/MILS3.SA/financials?p=MILS3.SA <Response [200]>
data-annual/MILS3.csv
https://finance.yahoo.com/quote/MLAS3.SA/financials?p=MLAS3.SA <Response [200]>
da

https://finance.yahoo.com/quote/SMTO3.SA/financials?p=SMTO3.SA <Response [200]>
data-annual/SMTO3.csv
https://finance.yahoo.com/quote/SOJA3.SA/financials?p=SOJA3.SA <Response [200]>
https://finance.yahoo.com/quote/SOMA3.SA/financials?p=SOMA3.SA <Response [200]>
data-annual/SOMA3.csv
https://finance.yahoo.com/quote/SQIA3.SA/financials?p=SQIA3.SA <Response [200]>
data-annual/SQIA3.csv
https://finance.yahoo.com/quote/STBP3.SA/financials?p=STBP3.SA <Response [200]>
data-annual/STBP3.csv
https://finance.yahoo.com/quote/SULA11.SA/financials?p=SULA11.SA <Response [200]>
data-annual/SULA11.csv
https://finance.yahoo.com/quote/SUZB3.SA/financials?p=SUZB3.SA <Response [200]>
data-annual/SUZB3.csv
https://finance.yahoo.com/quote/SYNE3.SA/financials?p=SYNE3.SA <Response [200]>
data-annual/SYNE3.csv
https://finance.yahoo.com/quote/TAEE11.SA/financials?p=TAEE11.SA <Response [200]>
data-annual/TAEE11.csv
https://finance.yahoo.com/quote/TAEE3.SA/financials?p=TAEE3.SA <Response [200]>
data-annual/TAEE3.